In [1]:
from bulk_load_to_database_between_dates import get_paths
from database_utils import reverse_extract_instrument_name, extract_instrument_name
from datetime import datetime

In [2]:
start_date = datetime(2023, 7, 27)
end_date = datetime(2023, 7, 27)

In [3]:
def instrument_name_to_glob_pattern(instrument_name: str) -> str:
    """Converts an instrument name to a glob pattern compatible with the e-Callisto 
    page containing the fits files at http://soleil.i4ds.ch/solarradio/data/2002-20yy_Callisto/
    """
    in_dirty = reverse_extract_instrument_name(instrument_name, include_number=True)
    print(in_dirty)
    split = in_dirty.split("-")
    number = split.pop()
    string = "-".join(split)

    return f"{string}*{number}.fit.gz" 

In [4]:
instrument_name_to_glob_pattern("URUGUAY_02")

URUGUAY-02


'URUGUAY*02.fit.gz'

In [6]:
paths = get_paths(start_date, end_date, instrument_regexr_pattern=instrument_name_to_glob_pattern("australia_assa_60"))

Australia-ASSA-60


fetching paths: 100%|██████████| 1/1 [00:00<00:00, 29.49it/s]


In [ ]:
# Create mapping from instrument name to table name
instrument_name_to_table_name = {k, v for 

In [92]:
import numpy as np
np.unique([extract_instrument_name(path) for path in paths["path"]])

array(['alaska_anchorage_59', 'alaska_cohoe_00', 'alaska_cohoe_01',
       'alaska_haarp_01', 'algeria_craag_59', 'almaty_58', 'almaty_59',
       'australia_assa_60', 'australia_lmro_59', 'austria_krumbach_10',
       'austria_oe3flb_55', 'austria_oe3flb_57', 'austria_unigraz_01',
       'egypt_01', 'germany_dlr_01', 'greenland_62', 'greenland_63',
       'humain_59', 'hurbanovo_59', 'india_gauri_59', 'india_iiserp_01',
       'india_nashik_59', 'india_ooty_58', 'india_ooty_59',
       'india_udaipur_01', 'indonesia_59', 'kasi_59', 'krim_00',
       'krim_02', 'mexart_59', 'mongolia_gobi_01', 'mongolia_ub_01',
       'newzealand_aut_59', 'norway_ny_aalesund_01',
       'norway_randaberg_01', 'roswell_nm_57', 'roswell_nm_58',
       'roswell_nm_59', 'southafrica_sansa_01', 'spain_alcala_03',
       'spain_peralejos_01', 'spain_peralejos_03', 'spain_siguenza_01',
       'swiss_blen5m_02', 'swiss_blen7m_01', 'swiss_hb9sct_01',
       'swiss_hb9sct_02', 'swiss_irsol_01', 'swiss_landschlac

In [81]:
from radiospectra.sources import CallistoSpectrogram

In [82]:
def get_last_spectrogram_from_paths_list(paths: dict[str, list[str]]) -> dict[str, str]:
    """Returns the last spectrogram for each instrument from a list of paths.
    """
    date_paths = zip(paths["date_changed"], paths["path"])
    # Make sure the paths are sorted by date
    last_path = sorted(date_paths, key=lambda x: x[0])[-1][1]

    print(last_path)

    spec = CallistoSpectrogram.read(last_path)
    return spec

In [83]:
spec = get_last_spectrogram_from_paths_list(paths)

/mnt/nas05/data01/radio/2002-20yy_Callisto/2021/01/01/AUSTRIA-UNIGRAZ_20210101_150002_01.fit.gz


In [7]:
import pandas as pd

In [19]:
spec = CallistoSpectrogram.read(paths['path'][-1])

In [23]:
spec.data.data

array([[184., 183., 184., ..., 183., 183., 183.],
       [172., 171., 172., ..., 172., 172., 171.],
       [181., 180., 181., ..., 181., 180., 181.],
       ...,
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan]])

In [28]:
df = pd.DataFrame(spec.data).dropna()

In [29]:
# Add meta data to the dataframe
for key, value in spec.header.items():
    df.attrs[key] = value

In [34]:
df.attrs['DATE']

'2021-01-01'

In [45]:
spec.header

SIMPLE  =                    T / file does conform to FITS standard             
BITPIX  =                    8 / number of bits per data pixel                  
NAXIS   =                    2 / number of data axes                            
NAXIS1  =                 3583 / length of data axis 1                          
NAXIS2  =                  200 / length of data axis 2                          
EXTEND  =                    T / FITS dataset may contain extensions            
COMMENT   FITS (Flexible Image Transport System) format defined in Astronomy and
COMMENT   Astrophysics Supplement Series v44/p363, v44/p371, v73/p359, v73/p365.
COMMENT   Contact the NASA Science Office of Standards and Technology for the   
COMMENT   FITS Definition document #100 and other FITS information.             
DATE    = '2021-01-01'         / Time of observation                            
CONTENT = '2021/01/01  Radio flux density, e-CALLISTO (AUSTRIA-UNIGRAZ)' / Title
ORIGIN  = 'OLG     '        